# Fetch dataset
I will be working with COCO Captions, which is a large-scale object detection, segmentation, and captioning dataset. It contains over 330k images, with more than 200k labeled images.

In [ ]:
%env KAGGLEHUB_CACHE=datasets
import kagglehub
import os
print("Importing KaggleHub...")
print(f'datasets path: {os.environ["KAGGLEHUB_DATASET_PATH"]}')
path = kagglehub.dataset_download("nikhil7280/coco-image-caption")
print("Path to dataset files:", path)

env: KAGGLEHUB_CACHE=datasets
Importing KaggleHub...
datasets path: datasets


100%|██████████| 12.8M/12.8M [00:01<00:00, 7.87MB/s]

Extracting zip of captions_train2014.json...


Path to dataset files: datasets/datasets/nikhil7280/coco-image-caption/versions/1/annotations_trainval2014/annotations/captions_train2014.json


In [ ]:
from torchvision.datasets import CocoCaptions
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
from torchvision.transforms import v2

transform = v2.Compose([
        v2.Resize((224, 224)),
        #v2.RandomCrop((224, 224), pad_if_needed=True, padding_mode='symmetric'),
        v2.ToTensor(),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

root = '.cache/datasets/coco/images/train2017'
data = CocoCaptions(root=root, annFile='.cache/datasets/coco/annotations/captions_train2017.json', transform=transform)
print(data[0][0].size())

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '.cache/datasets/coco/annotations/captions_train2017.json'

In [ ]:


model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.50.2"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_to

['a woman holding a bouquet of flowers in her hands']

In [11]:
predict_step(['./validation/1/iq2b6uifuacc1.jpeg'])

['an orange and white bowl of fruit on a tree']